# Yelp Data Merging

In [1]:
# Copied and pasted from my imports from project 3, 
# delete if we dont need(Aerika)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB, BernoulliNB

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords 

import regex as re

from scipy.stats import ttest_ind

In [2]:
file_paths = []
for file in os.listdir('../data'):
    if 'businesses2019' in file:
        file_paths.append('../data/'+file)

In [3]:
master_df = {
    'id': [],
    'latitude': [],
    'longitude': [],
    'price': [],
    'review_count': [],
    'rating': [],
    'zip_code': [],
    'city': [],
    'alias': [],
    'category': [],
}

master_df = pd.DataFrame(master_df)

In [4]:
for path in file_paths:
    
    master_df = pd.concat([master_df, pd.read_csv(path)])

master_df.head()

/Users/Work/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,chichen-itza-restaurant-los-angeles-3,"['mexican', 'sandwiches', 'soup']",Los Angeles,vC_6J_nGyf4J8xt-Vu6Shw,34.017440,-118.278300,$$,4.5,1190.0,90007.0
1,NaN,['childrensmuseums'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,['museums'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,figueroa-philly-cheese-steak-los-angeles-2,"['cheesesteaks', 'sandwiches', 'breakfast_brun...",Los Angeles,vfHJzF0ShYtwmotXE-0PiA,34.014196,-118.282417,$$,4.5,1076.0,90037.0
4,dirt-dog-los-angeles-4,"['hotdog', 'beerbar']",Los Angeles,0z23Jk7U_MpvtqKINPL2fA,34.028292,-118.275208,$,4.5,1900.0,90007.0


In [5]:
# Seeing shape of dataframe BEFORE dropping duplicates 

master_df.shape

(73995, 10)

In [6]:
# Dropping duplicates

master_df = master_df.sort_values(['alias',
                       'id', 'latitude', 'longitude', 'review_count'],
                      ascending = False).drop_duplicates(subset = ['id', 'alias', 'zip_code'],
                                                         keep ='first')

In [7]:
# Shape of df AFTER dropping duplicates

master_df.shape

(10077, 10)

In [8]:
# Resetting master_df index

master_df.reset_index(inplace=True)
master_df.drop(columns='index', inplace=True)

# NaNs in master_df

In [9]:
master_df.isna().sum()

alias            1
category         1
city             1
id               1
latitude         3
longitude        3
price            1
rating           1
review_count     1
zip_code        43
dtype: int64

In [10]:
# Dropping rows w/ prices that aren't $, $$, $$$, or $$$$.

data_drop_df = master_df.loc[(master_df['price'] != '$') & (master_df['price'] != '$$') & (master_df['price'] != '$$$') & (master_df['price'] != '$$$$')]
drop_indexes = list(data_drop_df.index)

master_df.drop(index=drop_indexes, inplace = True)
master_df.shape


(9910, 10)

In [11]:
# Businesses that don't have latitude/zipcodes = Food Trucks, Caterers --> dropped

food_trucks = master_df.loc[(master_df['latitude'].isna()) | (master_df['zip_code'].isna())].index.tolist()

master_df.drop(index = food_trucks, inplace = True)
master_df.shape


(9872, 10)

In [12]:
master_df.isna().sum()

alias           0
category        0
city            0
id              0
latitude        0
longitude       0
price           0
rating          0
review_count    0
zip_code        0
dtype: int64

# Replacing \\$ in price to numbers

In [13]:
master_df['price'] = master_df['price'].replace('$', 1)
master_df['price'] = master_df['price'].replace('$$', 2)
master_df['price'] = master_df['price'].replace('$$$', 3)
master_df['price'] = master_df['price'].replace('$$$$', 4)

In [14]:
master_df.head(2)

,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,いざかや-おせん-izakaya-osen-los-angeles,"['sushi', 'izakaya', 'seafood']",Los Angeles,us0WnDOySVXXXwCqs0AaCw,34.083192,-118.273245,2,4.5,651.0,90026.0
1,ô-banh-mi-los-angeles,"['vietnamese', 'sandwiches', 'coffee']",Los Angeles,XRbyfQZ7nWwJCOjRC8EW-g,34.098450,-118.275100,1,4.0,111.0,90027.0


In [114]:
master_df = master_df.reset_index(drop=True)

# Dropping Rows 
- zipcodes that don't start with '9'

In [130]:
notin_la = []
for i in range(len(master_df['zip_code'])):
    if master_df['zip_code'][i].startswith('9') == False:
        notin_la.append(master_df['zip_code'].index[i])

In [131]:
notin_la

[2211,
 2753,
 3269,
 3361,
 3362,
 3367,
 3386,
 3387,
 3400,
 3401,
 3404,
 5155,
 5687,
 8726]

In [134]:
master_df.shape

(9872, 10)

In [136]:
master_df = master_df.drop(index=notin_la).reset_index(drop=True)

## Save the cleaned data to a CSV

In [15]:
# Don't uncomment this unless you actually want to create a new csv

# master_df.to_csv('../data/master.csv')